In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(r'/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.figure(figsize=(20,25), facecolor='white')
plotnumber = 1

for column in df:
    if plotnumber<=9 :     # as there are 9 columns in the data
        ax = plt.subplot(3,3,plotnumber)
        sns.distplot(df[column])
        plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.show()

In [ ]:
df['BMI'] = df['BMI'].replace(0,df['BMI'].mean())
df['BloodPressure'] = df['BloodPressure'].replace(0,df['BloodPressure'].mean())
df['Glucose'] = df['Glucose'].replace(0,df['Glucose'].mean())
df['Insulin'] = df['Insulin'].replace(0,df['Insulin'].mean())
df['SkinThickness'] = df['SkinThickness'].replace(0,df['SkinThickness'].mean())

In [ ]:
plt.figure(figsize=(20,25), facecolor='white')
plotnumber = 1

for column in df:
    if plotnumber<=9 :
        ax = plt.subplot(3,3,plotnumber)
        sns.distplot(df[column])
        plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.boxplot(data=df, width= 0.5,ax=ax,  fliersize=3)

In [ ]:
q = df['Pregnancies'].quantile(0.98)
# we are removing the top 2% data from the Pregnancies column
data_cleaned = df[df['Pregnancies']<q]
q = data_cleaned['BMI'].quantile(0.99)
# we are removing the top 1% data from the BMI column
data_cleaned  = data_cleaned[data_cleaned['BMI']<q]
q = data_cleaned['SkinThickness'].quantile(0.99)
# we are removing the top 1% data from the SkinThickness column
data_cleaned  = data_cleaned[data_cleaned['SkinThickness']<q]
q = data_cleaned['Insulin'].quantile(0.95)
# we are removing the top 5% data from the Insulin column
data_cleaned  = data_cleaned[data_cleaned['Insulin']<q]
q = data_cleaned['DiabetesPedigreeFunction'].quantile(0.99)
# we are removing the top 1% data from the DiabetesPedigreeFunction column
data_cleaned  = data_cleaned[data_cleaned['DiabetesPedigreeFunction']<q]
q = data_cleaned['Age'].quantile(0.99)
# we are removing the top 1% data from the Age column
data_cleaned  = data_cleaned[data_cleaned['Age']<q]

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.boxplot(data=data_cleaned, width= 0.5,ax=ax,  fliersize=3)

In [ ]:
from scipy import stats     
import numpy as np
z = np.abs(stats.zscore(data_cleaned))      #Using Z-score for removing some more outliers
print(z)

In [ ]:
threshold = 3
print(np.where(z > 3))

In [ ]:
data1 = data_cleaned[(z < 3).all(axis=1)]

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.boxplot(data=data1, width= 0.5,ax=ax,  fliersize=3)

In [ ]:
data1.shape

In [ ]:
y = data1['Outcome']
X = data1.drop('Outcome',axis=1)

In [ ]:
plt.figure(figsize=(20,25), facecolor='white')
plotnumber = 1

for column in data1:
    if plotnumber<=9 :
        ax = plt.subplot(3,3,plotnumber)
        sns.distplot(data1[column])
        plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.show()

In [ ]:
plt.figure(figsize=(20,25), facecolor='white')
plotnumber = 1

for column in X:
    if plotnumber<=9 :
        ax = plt.subplot(3,3,plotnumber)
        sns.stripplot(y,X[column])
    plotnumber+=1
plt.tight_layout()

In [ ]:
#Standard-Scaling
from sklearn.preprocessing import StandardScaler   
scalar = StandardScaler()                               
X_scaled = scalar.fit_transform(X)

In [ ]:
#Check for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif["vif"] = [variance_inflation_factor(X_scaled,i) for i in range(X_scaled.shape[1])]
vif["Features"] = X.columns

#let's check the values
vif

No multicollinearity as such(Since vif values are below 5)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_scaled,y, test_size= 0.2, random_state = 60)

# Logistic Regression

In [ ]:
from sklearn.linear_model  import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(x_train,y_train)

In [ ]:
y_pred = log_reg.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
accuracy = accuracy_score(y_test,y_pred)
accuracy

In [ ]:
# Confusion Matrix
conf_mat = confusion_matrix(y_test,y_pred)
conf_mat

In [ ]:
true_positive = conf_mat[0][0]
false_positive = conf_mat[0][1]
false_negative = conf_mat[1][0]
true_negative = conf_mat[1][1]

In [ ]:
# Breaking down the formula for Accuracy
Accuracy = (true_positive + true_negative) / (true_positive +false_positive + false_negative + true_negative)
Accuracy

In [ ]:
# Precison
Precision = true_positive/(true_positive+false_positive)
Precision

In [ ]:
# Recall
Recall = true_positive/(true_positive+false_negative)
Recall

In [ ]:
# F1 Score
F1_Score = 2*(Recall * Precision) / (Recall + Precision)
F1_Score

In [ ]:
# Area Under Curve
auc = roc_auc_score(y_test, y_pred)
auc

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

In [ ]:
clf.score(x_train,y_train)

In [ ]:
clf.score(x_test,y_test)

Seems the model is highly overfitting, lets perform hyperparameter tuning using Grid Search CV and see whether the accuracy improves.

In [ ]:
grid_param = {
    'criterion': ['gini', 'entropy'],
    'max_depth' : range(2,32,1),
    'min_samples_leaf' : range(1,10,1),
    'min_samples_split': range(2,10,1),
    'splitter' : ['best', 'random']
    
}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator=clf,
                     param_grid=grid_param,
                     cv=5,
                    n_jobs =-1)

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
best_parameters = grid_search.best_params_
print(best_parameters)

In [ ]:
clf = DecisionTreeClassifier(criterion = 'entropy', max_depth =10, min_samples_leaf= 9, min_samples_split= 2, splitter ='random')
clf.fit(x_train,y_train)

In [ ]:
clf.score(x_test,y_test)

Accuracy improved marginally.

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
#Area under the curve
auc = roc_auc_score(y_test, y_pred)
auc

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rand_clf = RandomForestClassifier(random_state=6)

In [ ]:
rand_clf.fit(x_train,y_train)

In [ ]:
rand_clf.score(x_test,y_test)

Performing hyperparameter tuning

In [ ]:
grid_param = {
    "n_estimators" : [90,100,115],
    'criterion': ['gini', 'entropy'],
    'max_depth' : range(2,20,2),
    'min_samples_leaf' : range(1,10,1),
    'min_samples_split': range(2,10,1),
    'max_features' : ['auto','log2']
}

In [ ]:
grid_search = GridSearchCV(estimator=rand_clf,param_grid=grid_param,cv=5,n_jobs =-1,verbose = 3)

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_params_

In [ ]:
rand_clf = RandomForestClassifier(criterion= 'entropy',
 max_depth = 10,
 max_features = 'auto',
 min_samples_leaf = 3,
 min_samples_split= 8,
 n_estimators = 100,random_state=6)

In [ ]:
rand_clf.fit(x_train,y_train)

Getting almost the same accuracy after hyperparameter tuning, maybe it needed more parameters

In [ ]:
rand_clf.score(x_test,y_test)

In [ ]:
y_pred = rand_clf.predict(x_test)

In [ ]:
auc = roc_auc_score(y_test, y_pred)
auc

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(x_train,y_train)

In [ ]:
knn.score(x_train,y_train)

In [ ]:
knn.score(x_test,y_test)

Performing hyperparameter tuning

In [ ]:
param_grid = { 'algorithm' : ['ball_tree', 'kd_tree', 'brute'],
               'leaf_size' : [18,20,25,27,30,32,34],
               'n_neighbors' : [3,5,7,9,10,11,12,13,15,17,19]
              }

In [ ]:
gridsearch = GridSearchCV(knn, param_grid,verbose=3)

In [ ]:
gridsearch.fit(x_train,y_train)

In [ ]:
gridsearch.best_params_

In [ ]:
knn = KNeighborsClassifier(algorithm = 'ball_tree', leaf_size =18, n_neighbors =13)

In [ ]:
knn.fit(x_train,y_train)

In [ ]:
knn.score(x_train,y_train)

In [ ]:
knn.score(x_test,y_test)

Seems before hyperparameter tuning, the model was overfitting, now it is better.


In [ ]:
y_pred = knn.predict(x_test)

In [ ]:
auc = roc_auc_score(y_test, y_pred)
auc

# XGBoost

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(objective='binary:logistic')
model.fit(x_train,y_train)

In [ ]:
# cheking training accuracy
y_pred = model.predict(x_train)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_train,predictions)
accuracy

In [ ]:
# cheking initial test accuracy
y_pred = model.predict(x_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test,predictions)
accuracy

The model is highly overfitting. Need to perform hyperparameter tuning

In [ ]:
param_grid={
   
    'learning_rate':[1,0.5,0.1,0.01,0.001],
    'max_depth': [3,5,10,20],
    'n_estimators':[10,50,100,200]
    
}

In [ ]:
grid= GridSearchCV(XGBClassifier(objective='binary:logistic'),param_grid, verbose=3)

In [ ]:
grid.fit(x_train,y_train)

In [ ]:
grid.best_params_

In [ ]:
new_model=XGBClassifier(learning_rate=0.01, max_depth= 3, n_estimators= 200)
new_model.fit(x_train, y_train)

In [ ]:
y_pred_new = new_model.predict(x_test)
predictions_new = [round(value) for value in y_pred_new]
accuracy_new = accuracy_score(y_test,predictions_new)
accuracy_new

The accuracy has increased for the test dataset.

In [ ]:
auc = roc_auc_score(y_test, y_pred_new)
auc

# ** Considering the accuracy and AUC scores of all the models taken here, Random Forest Classifier and XGBoost Classifier models perform the best